In [1]:
import numpy as np
import math
from enum import Enum
from queue import PriorityQueue
from queue import Queue

In [2]:
grid_8 = np.array([
    [1,2,5],
    [3,4,0],
    [6,7,8],
])

In [3]:
tuple_8 = tuple(grid_8)
print(tuple_8)

(array([1, 2, 5]), array([3, 4, 0]), array([6, 7, 8]))


In [4]:
grid_check = np.array([
    [0,1,2],
    [3,4,5],
    [6,7,8],
])

In [13]:
list_8 = grid_check.flatten().tolist()
list_8 == [0,1,2,3,4,5,6,7,8]
print(tuple(list_8))

(0, 1, 2, 3, 4, 5, 6, 7, 8)


In [8]:
index = np.where(grid_8 == 0)
# print(index)
x = int(index[0])
y = int(index[1])
print(x)
print(y)
current_node = (x,y)
print(current_node)
print(type(current_node))

1
2
(1, 2)
<class 'tuple'>


In [9]:
def valid_actions(grid, current_node):
    """
    Returns a list of valid actions given a grid and 0 node.
    """
    index = np.where(grid_8 == 0)
    valid = [Action.UP, Action.LEFT, Action.RIGHT, Action.DOWN]
    n, m = grid_8.shape[0] - 1, grid_8.shape[1] - 1
    x, y = int(index[0]),int(index[1])
    
    
    # check if the node is off the grid or it's an obstacle
    
    if x - 1 < 0 or grid_8[x-1, y] == 1:
        valid.remove(Action.UP)
    if x + 1 > n or grid_8[x+1, y] == 1:
        valid.remove(Action.DOWN)
    if y - 1 < 0 or grid_8[x, y-1] == 1:
        valid.remove(Action.LEFT)
    if y + 1 > m or grid_8[x, y+1] == 1:
        valid.remove(Action.RIGHT)     
    return valid

In [17]:
#  initialize a set() object for your visited list and add the start location to it
visited = set()
# print(list_8)
visited.add(tuple(list_8))
visited.add((3,5,6,1,0,4,7,8,2))
print(visited)

{(3, 5, 6, 1, 0, 4, 7, 8, 2), (0, 1, 2, 3, 4, 5, 6, 7, 8)}


In [18]:
start = (1,2,5,3,4,0,6,7,8)

In [19]:
class Action(Enum):

    LEFT = (0, -1, 1)
    RIGHT = (0, 1, 1)
    UP = (-1, 0, 1)
    DOWN = (1, 0, 1)
             
    @property
    def cost(self):
        return self.value[2]
    
    @property
    def delta(self):
        return (self.value[0], self.value[1])  

In [20]:
def valid_actions(list):
    """
    Returns a list of valid actions given a grid and 0 node.
    """
    grid = np.array(list).reshape(3,-1)
    
    index = np.where(grid == 0)
    valid = [Action.UP, Action.LEFT, Action.RIGHT, Action.DOWN]
    
    n, m = grid.shape[0] - 1, grid.shape[1] - 1
    x, y = int(index[0]),int(index[1])

    
    
    # check if the node is off the grid or it's an obstacle
    
    if x - 1 < 0 or grid[x-1, y] == 1:
        valid.remove(Action.UP)
    if x + 1 > n or grid[x+1, y] == 1:
        valid.remove(Action.DOWN)
    if y - 1 < 0 or grid[x, y-1] == 1:
        valid.remove(Action.LEFT)
    if y + 1 > m or grid[x, y+1] == 1:
        valid.remove(Action.RIGHT)     
    return valid

In [21]:
a = valid_actions(start)
print(a)
print(a[0].delta)

[<Action.UP: (-1, 0, 1)>, <Action.LEFT: (0, -1, 1)>, <Action.DOWN: (1, 0, 1)>]
(-1, 0)


In [92]:
# initialize a Queue() object and add the start location to it:
q = Queue()
q.put(start)

In [93]:
#  initialize a set() object for your visited list and add the start location to it
visited = set()
visited.add(start)

In [94]:
# define an empty dictionary, where you'll record how you moved through the grid and a goal location, 
branch = {}
goal = (0,1,2,3,4,5,6,7,8)

In [95]:
start = (1,2,5,3,4,0,6,7,8)
da = a[0].delta
# print(da[0])
# print(da[1])
grid = np.array(start).reshape(3,-1)
print(grid)

index = np.where(grid == 0)
# print(index[0])
# print(index[1])

x, y = int(index[0]),int(index[1]) #node of 0

grid[x+da[0],y+da[1]],grid[x,y] = grid[x,y],grid[x+da[0],y+da[1]]
next_grid = grid
print(next_grid)


[[1 2 5]
 [3 4 0]
 [6 7 8]]
[[1 2 0]
 [3 4 5]
 [6 7 8]]


In [97]:
current_node = start

for action in a:
# for action in valid_actions:
    # delta of performing the action

    da = action.delta
    
    grid = np.array(start).reshape(3,-1)
    index = np.where(grid == 0)
    x, y = int(index[0]),int(index[1])
    
    grid[x+da[0],y+da[1]],grid[x,y] = grid[x,y],grid[x+da[0],y+da[1]]
    next_node = tuple(grid.flatten().tolist())


    # Check if the new node has been visited before.
    # If the node has not been visited you will need to
    # 1. Mark it as visited
    # 2. Add it to the queue
    # 3. Add how you got there to branch
    if next_node not in visited:                
        visited.add(next_node)               
        q.put(next_node)          
        branch[next_node] = (current_node, action) 

print(q.queue)
print(visited)
print(branch) 

deque([(1, 2, 5, 3, 4, 0, 6, 7, 8), (1, 2, 0, 3, 4, 5, 6, 7, 8), (1, 2, 5, 3, 0, 4, 6, 7, 8), (1, 2, 5, 3, 4, 8, 6, 7, 0)])
{(1, 2, 0, 3, 4, 5, 6, 7, 8), (1, 2, 5, 3, 4, 0, 6, 7, 8), (1, 2, 5, 3, 4, 8, 6, 7, 0), (1, 2, 5, 3, 0, 4, 6, 7, 8)}
{(1, 2, 0, 3, 4, 5, 6, 7, 8): ((1, 2, 5, 3, 4, 0, 6, 7, 8), <Action.UP: (-1, 0, 1)>), (1, 2, 5, 3, 0, 4, 6, 7, 8): ((1, 2, 5, 3, 4, 0, 6, 7, 8), <Action.LEFT: (0, -1, 1)>), (1, 2, 5, 3, 4, 8, 6, 7, 0): ((1, 2, 5, 3, 4, 0, 6, 7, 8), <Action.DOWN: (1, 0, 1)>)}


In [91]:
q.queue.clear()

In [ ]:
def bfs(grid,start,goal):
    
        